In [44]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

import pandas as pd
from tensorflow.keras.models import load_model
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
model = load_model("chatbot.h5")

def clean_up_sentense(sentense):
    sentense_words = nltk.word_tokenize(sentense)
    sentense_words = [lemmatizer.lemmatize(word) for word in sentense_words]
    return sentense_words

def bag_of_words(sentense):
    sentense_words = clean_up_sentense(sentense)
    bag = [0] * len(words)
    for w in sentense_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    Error_threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > Error_threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result   

print("Go! Bot is running")  

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

Go! Bot is running
I am sleeping with your mom
I am sleeping with your mom
Sad to see you go :(
I am sleeping with your mom
Bair likes dicks


KeyboardInterrupt: Interrupted by user